# The Yelp Star Predicter

## Part 1: Refining the dataset


We pulled out the data from Yelp http://www.yelp.com/dataset_challenge which contains:

1.6M reviews and 500K tips by 366K users for 61K businesses, 481K business attributes, e.g., hours, parking availability, ambience, social network of 366K users for a total of 2.9M social edges, aggregated check-ins over time for each of the 61K businesses.



**First challenge: **

First issue we encountered: the file is not JSON but each lines is written in the JSON format. 
Solution: we looped through each lines to make it into a csv using the following code:

    import pandas as pd
    import json
    lines = open("untitled.txt","r").readlines()
    elements = [json.loads(line) for line in lines]
    for element in elements:
    element['funny'] = element['votes']['funny']
    element['useful'] = element['votes']['useful']
    element['cool'] = element['votes']['cool']
    df = pd.DataFrame(elements)``
    
    import csv
    import sys
    reload(sys)
    sys.setdefaultencoding("utf-8")
    with open("yelp_academic_dataset_review.json","r") as yelpfile:
    with open("results.csv","w") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['votes', 'user_id', 'review_id', 'text', 'business_id', 'stars', 'date', 'type', 'votes_funny','votes_cool','votes_useful'])
    writer.writeheader()
       
    for line in yelpfile:
    json_line = json.loads(line)
    json_line['votes_funny'] = json_line['votes']['funny']
    json_line['votes_useful'] = json_line['votes']['useful']
    json_line['votes_cool'] = json_line['votes']['cool']
    writer.writerow(json_line)




**Second challenge**

Question: How could we look into the data relevant to Las Vegas only? 

Solution: We realized that in order to naviguate through the data we would need to import our newly created CSV file into a database.  


**All about categorie…**

Puting the c ategories into categories. Example: we have a column called vote. Some of these votes are defined as followed: ‘vote funny’, ‘vote cool’ or ‘vote happy’). Instead of having this all gathered into the categorie ‘Vote’ we created additional columns so that it would be easier to filtre categories down the line.

How did we do that? We first created smaller files by taking the fiest 10 rows of the dataset in order to create smaller version dataset. Which allowed to try out our new dataset with our new columns.

In other words =>We changed the code to upload different features (business idea, full adress ect)




**Creating the dataset**

Issue faced: list indices must be integers not str when trying to make all files csv before creating dataset. Hence we had to save the list to a CSV file by closing the csv and the input file for each file.This process involeved: flattering the dataset first.  Flattenning some of the data.




** Merging the data using SQL and Pandas**

In [3]:
import psycopg2

con = psycopg2.connect(database="utility", user="lede",password="columbia", host="utility.c1erymiua9dx.us-east-1.rds.amazonaws.com")
cur = con.cursor()

In [4]:
cur.execute("SELECT * FROM lede.yelp_review LIMIT 10")
cur.fetchone()

("{u'funny': 0, u'useful': 0, u'cool': 0}",
 'aJeQbvD9rEpmfHMvw3cF_g',
 '5hhJc35OX1Ykqs3wLbXX5g',
 'Market Grille represents everything that you could ask for in a restaurant. Quality food, nice ambience, great service and good value.Although the emphasis is on Greek specialties, the mix can satisfy almost anyone, especially now that Italian food is available. If you are looking for burgers and pizza, grow up, get over it and try something different. The soups are extraordinary especially the Mushroom Pumpkin flavored with cinnamon. For desert you will find the best rice pudding along with the most delicious Baklava Cheesecake anywhere. This is an old style family enterprise and it shows. One suggestion........go there hungry.......trust me.',
 '2Fn5-x5IX7CfhCKFYaZGyA',
 5,
 '2014-12-13',
 'review',
 0,
 0,
 0)

For the merging part we need to make sure that we have no duplicate columns. So we use list comprehensions to put all the things together. 

In [5]:
con.rollback()
cur.execute("Select * FROM lede.yelp_review LIMIT 1")
rcols = [d[0] for d in cur.description]

con.rollback()
cur.execute("Select * FROM lede.yelp_business LIMIT 1")
bcols = [d[0] for d in cur.description]

con.rollback()
cur.execute("Select * FROM lede.yelp_user LIMIT 1")
ucols = [u[0] for u in cur.description]

cols = [ "r."+c  for c in rcols ] + [ "b."+c  for c in bcols  if c not in rcols ] + ["u."+ u for u in ucols if u not in rcols and u not in bcols]
#cols = "r.*,"+",".join( [ "b."+c  for c in bcols  if c not in rcols ] )
cols

['r.votes',
 'r.user_id',
 'r.review_id',
 'r.review_text',
 'r.business_id',
 'r.stars',
 'r.review_date',
 'r.review_type',
 'r.votes_funny',
 'r.votes_cool',
 'r.votes_useful',
 'b.city',
 'b.review_count',
 'b.business_name',
 'b.neighborhoods',
 'b.bus_type',
 'b.full_address',
 'b.hours',
 'b.state',
 'b.longitude',
 'b.latitude',
 'b.attributes',
 'b.open',
 'b.categories',
 'u.index_num',
 'u.average_stars',
 'u.compliments',
 'u.elite',
 'u.fans',
 'u.friends',
 'u.user_name',
 'u.type_acccount',
 'u.yelping_since']

In [6]:
con.rollback()
cur.execute("DROP TABLE yelp_predict2")
cur.execute(
    "CREATE TABLE yelp_predict2 AS (SELECT "+",".join( cols )+" FROM lede.yelp_review r INNER JOIN lede.yelp_business b ON r.business_id = b.business_id INNER JOIN lede.yelp_user u ON u.user_id = r.user_id WHERE b.city = 'Las Vegas') "
)
con.commit()

In [8]:
con.rollback()
cur.execute("SELECT * FROM yelp_predict2 LIMIT 10")
firstten = list(cur)
import pandas as pd
yelp_complete = pd.DataFrame(firstten,columns=cols)
yelp_complete

,r.votes,r.user_id,r.review_id,r.review_text,r.business_id,r.stars,r.review_date,r.review_type,r.votes_funny,r.votes_cool,...,b.categories,u.index_num,u.average_stars,u.compliments,u.elite,u.fans,u.friends,u.user_name,u.type_acccount,u.yelping_since
0,"{u'funny': 1, u'useful': 4, u'cool': 1}",XjfKBr96h6GgrfkR_q6gIA,b7crq9zDm0Zfr5jpnRbaJw,I'm surprised that I didn't write up a review ...,LKvIwGxtj8z1F0F_pF3v0g,4,2011-12-22,review,1,1,...,"[Shopping, Toy Stores]",None,3.96,"{""profile"": 3, ""funny"": 5, ""cute"": 1, ""plain"":...","[2010, 2011, 2012, 2013, 2014, 2015]",34,"[U0QGtmk81ca5CK0Alh_x_w, i8KMQX7nekLSCzb_DVb6y...",Margret,user,2010-05
1,"{u'funny': 0, u'useful': 2, u'cool': 0}",glRXVWWD6x1EZKfjJawTOg,jH0MPbfU96hYEie7WdQC2w,I don't have a kid and I don't like branded an...,LKvIwGxtj8z1F0F_pF3v0g,3,2013-01-22,review,0,0,...,"[Shopping, Toy Stores]",None,3.79,"{""profile"": 49, ""funny"": 261, ""cute"": 24, ""pla...","[2010, 2011, 2012, 2013, 2014, 2015]",87,"[rpOyqD_893cqmDAtJLbdog, zTWH9b_ItSdLOK9ypeFOI...",Jade,user,2009-04
2,"{u'funny': 1, u'useful': 4, u'cool': 1}",PTIR5xjim9PKnzoquXdD7A,4qeLTDUcJKPDF9cnPYjY8w,Damn delicious. Do yourself a favor and try on...,2e2e7WgqU1BnpxmQL5jbfw,5,2008-01-29,review,1,1,...,"[Sandwiches, Restaurants]",None,3.73,"{""funny"": 3, ""plain"": 4, ""writer"": 3, ""list"": ...",[2008],8,"[y4F30s5qp4VBInqmt4Upyg, AaZdXn0I6F5bdIVwGpxdD...",Vanessa,user,2006-09
3,"{u'funny': 0, u'useful': 0, u'cool': 0}",3MbsABpy7ocSoInD87GANg,-4elpu3SGRSChGccQii1GA,Excellent sandwiches...wish they have a place ...,2e2e7WgqU1BnpxmQL5jbfw,5,2009-06-07,review,0,0,...,"[Sandwiches, Restaurants]",None,3.58,"{""plain"": 1, ""hot"": 1}",[],0,"[WmAyExqSWoiYZ5XEqpk_Uw, UuZjstcJYy8vi84wzYQhZ...",Justin,user,2009-06
4,"{u'funny': 0, u'useful': 1, u'cool': 1}",17LPTsu4RgqTZ3SxsGzzxw,8IRSdQyI0qJoWbT_t6Z0Uw,My family told me how amazing Earl of Sandwich...,2e2e7WgqU1BnpxmQL5jbfw,4,2009-11-25,review,0,1,...,"[Sandwiches, Restaurants]",None,3.82,"{""profile"": 8, ""funny"": 56, ""plain"": 120, ""wri...","[2010, 2011, 2012, 2013, 2014, 2015]",52,"[zTWH9b_ItSdLOK9ypeFOIw, LroJKi7_BWqDT7lbHvt6k...",Rob,user,2008-01
5,"{u'funny': 1, u'useful': 2, u'cool': 2}",glRXVWWD6x1EZKfjJawTOg,pqBEb_MkUUMG-b7JzBVPuQ,"When I worked in planet hollywood, my food cho...",2e2e7WgqU1BnpxmQL5jbfw,5,2010-02-28,review,1,2,...,"[Sandwiches, Restaurants]",None,3.79,"{""profile"": 49, ""funny"": 261, ""cute"": 24, ""pla...","[2010, 2011, 2012, 2013, 2014, 2015]",87,"[rpOyqD_893cqmDAtJLbdog, zTWH9b_ItSdLOK9ypeFOI...",Jade,user,2009-04
6,"{u'funny': 0, u'useful': 0, u'cool': 0}",DmQgSk0K4H4GbUOzsAutdw,vEUrkelR_zTLDIfFgTbuBQ,EARL OF MUTHAF'ING SANDWICH FOR THE MUTHAF'ING...,2e2e7WgqU1BnpxmQL5jbfw,5,2010-06-24,review,0,0,...,"[Sandwiches, Restaurants]",None,3.75,"{""profile"": 1, ""funny"": 23, ""cute"": 4, ""plain""...","[2009, 2010, 2011]",4,"[GJYJX4SujVj3BR8v2F9PDQ, w6Vv-kldGpmvSGqXvTbAd...",Jeffrey,user,2007-12
7,"{u'funny': 2, u'useful': 9, u'cool': 4}",kGgAARL2UmvCcTRfiscjug,ArZ22Ni0z-rsYCz5zF6NUw,Any restaurant with 4.5 Stars after hundreds o...,2e2e7WgqU1BnpxmQL5jbfw,5,2010-06-27,review,2,4,...,"[Sandwiches, Restaurants]",None,3.62,"{""profile"": 112, ""cute"": 42, ""funny"": 427, ""pl...",[2010],164,"[rpOyqD_893cqmDAtJLbdog, 8J4IIYcqBlFch8T90N923...",J,user,2007-03
8,"{u'funny': 0, u'useful': 1, u'cool': 0}",8JC-Yb3UDUv2FUl5ym1nVg,ODdZ3fuzFVBDk50krUu67Q,"Ever had a late night, and somehow you found y...",2e2e7WgqU1BnpxmQL5jbfw,5,2010-08-01,review,0,0,...,"[Sandwiches, Restaurants]",None,4.06,"{""profile"": 7, ""cute"": 4, ""funny"": 58, ""plain""...","[2011, 2012, 2013, 2014]",35,"[rpOyqD_893cqmDAtJLbdog, DglSIRnl69GUqHvVnG3Qa...",Dave,user,2007-12
9,"{u'funny': 0, u'useful': 0, u'cool': 0}",UMxMwut-4yxEhe19G9_rOg,pEMemhyREpKz_ma8WTYJmA,"After reading all the reviews here, I decided ...",2e2e7WgqU1BnpxmQL5jbfw,3,2010-08-21,review,0,0,...,"[Sandwiches, Restaurants]",None,3.76,"{""note"": 1, ""photos"": 1, ""plain"": 2, ""cool"": 1}",[],0,[WKR1O8p2

In [61]:
con.rollback()

#cur.execute("SELECT * FROM yelp_predict2 WHERE categories[2] = 'Restaurants'  LIMIT 10;")
cur.execute("SELECT * FROM yelp_predict2 WHERE ('Restaurants' = any(categories));")
#print cur.fetchall()

res = list(cur)
import pandas as pd
resdf = pd.DataFrame(res,columns=cols)

In [62]:
resdf.to_csv("vegas_restaurants_test.csv")
